求解方法分类
> 分支定界法 -- 可求纯嚯混合整数线性规划  
> 割平面法 -- 可求纯或混合整数线性规划  
> 隐枚举法 -- 求解“0-1”整数规划  
>+ 过滤隐枚举法  
>+ 分支隐枚举法  

> 匈牙利法 -- 解决指派问题（“0-1”规划特殊情形）  
> 蒙特卡洛法 -- 求解各种类型规划  

## 例2.1
min z = x1 + x2  
s.t.
- 2x1 + 4x2 = 5
- x1, x2 >= 0

In [7]:
from scipy.optimize import linprog

z = [1, 1]
a = [[2,4]]
b = [5]

bounds = [(0, None), (0, None)]

res = linprog(z, A_eq=a, b_eq=b, bounds=bounds, method='simplex')
print(res)

     con: array([0.])
     fun: 1.25
 message: 'Optimization terminated successfully.'
     nit: 1
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([0.  , 1.25])


## 0-1 整数规划 - 分支界定法

In [8]:
from scipy.optimize import linprog
import numpy as np
import math
import sys
from queue import Queue


class ILP():
    def __init__(self, c, A_ub, b_ub, A_eq, b_eq, bounds):
        # 全局参数
        self.LOWER_BOUND = -sys.maxsize
        self.UPPER_BOUND = sys.maxsize
        self.opt_val = None
        self.opt_x = None
        self.Q = Queue()

        # 这些参数在每轮计算中都不会改变
        self.c = -c
        self.A_eq = A_eq
        self.b_eq = b_eq
        self.bounds = bounds

        # 首先计算一下初始问题
        r = linprog(-c, A_ub, b_ub, A_eq, b_eq, bounds)

        # 若最初问题线性不可解
        if not r.success:
            raise ValueError('Not a feasible problem!')

        # 将解和约束参数放入队列
        self.Q.put((r, A_ub, b_ub))

    def solve(self):
        while not self.Q.empty():
            # 取出当前问题
            res, A_ub, b_ub = self.Q.get(block=False)

            # 当前最优值小于总下界，则排除此区域
            if -res.fun < self.LOWER_BOUND:
                continue

            # 若结果 x 中全为整数，则尝试更新全局下界、全局最优值和最优解
            if all(list(map(lambda f: f.is_integer(), res.x))):
                if self.LOWER_BOUND < -res.fun:
                    self.LOWER_BOUND = -res.fun

                if self.opt_val is None or self.opt_val < -res.fun:
                    self.opt_val = -res.fun
                    self.opt_x = res.x

                continue

            # 进行分枝
            else:
                # 寻找 x 中第一个不是整数的，取其下标 idx
                idx = 0
                for i, x in enumerate(res.x):
                    if not x.is_integer():
                        break
                    idx += 1

                # 构建新的约束条件（分割
                new_con1 = np.zeros(A_ub.shape[1])
                new_con1[idx] = -1
                new_con2 = np.zeros(A_ub.shape[1])
                new_con2[idx] = 1
                new_A_ub1 = np.insert(A_ub, A_ub.shape[0], new_con1, axis=0)
                new_A_ub2 = np.insert(A_ub, A_ub.shape[0], new_con2, axis=0)
                new_b_ub1 = np.insert(b_ub, b_ub.shape[0], -math.ceil(res.x[idx]), axis=0)
                new_b_ub2 = np.insert(b_ub, b_ub.shape[0], math.floor(res.x[idx]), axis=0)

                # 将新约束条件加入队列，先加最优值大的那一支
                r1 = linprog(self.c, new_A_ub1, new_b_ub1, self.A_eq, self.b_eq, self.bounds)
                r2 = linprog(self.c, new_A_ub2, new_b_ub2, self.A_eq, self.b_eq, self.bounds)
                if not r1.success and r2.success:
                    self.Q.put((r2, new_A_ub2, new_b_ub2))
                elif not r2.success and r1.success:
                    self.Q.put((r1, new_A_ub1, new_b_ub1))
                elif r1.success and r2.success:
                    if -r1.fun > -r2.fun:
                        self.Q.put((r1, new_A_ub1, new_b_ub1))
                        self.Q.put((r2, new_A_ub2, new_b_ub2))
                    else:
                        self.Q.put((r2, new_A_ub2, new_b_ub2))
                        self.Q.put((r1, new_A_ub1, new_b_ub1))

In [9]:
def test2():
    """ 此测试的真实最优解为 [2, 4] """
    c = np.array([3, 13])
    A = np.array([[2, 9], [11, -8]])
    b = np.array([40, 82])
    Aeq = None
    beq = None
    bounds = [(0, None), (0, None)]

    solver = ILP(c, A, b, Aeq, beq, bounds)
    solver.solve()

    print("Test 2's result:", solver.opt_val, solver.opt_x)
    print("Test 2's true optimal x: [2, 4]\n")

In [10]:
test2()

Test 2's result: 58.0 [2. 4.]
Test 2's true optimal x: [2, 4]



## 非线性规划问题求解

In [11]:
from scipy.optimize import minimize
import numpy as np

def fun():
    v = lambda a: - a[0]**2 - a[1]**2 - 3*a[2]*a[2] - 4*a[3]*a[3] - 2*a[4]*a[4] + 8*a[0] + 2*a[1] + 3*a[2] + a[3] + 2*a[4]
    return v

def con():
    # 约束条件，分为eq和ineq
    # eq表示函数等于0，ineq表示表达式大于等于0
    cons = ({'type':'ineq', 'fun':lambda a:400-sum(a)},
            {'type':'ineq', 'fun':lambda a:800-a[0]-2*a[1]-2*a[2]-a[3]-6*a[4]},
            {'type':'ineq', 'fun':lambda a:200-2*a[0]-a[1]-6*a[2]},
            {'type':'ineq', 'fun':lambda a:200-a[2]-a[3]-5*a[4]})
    return cons

In [12]:
lst = [50,2,93,92,20]
cons = con()
bounds = ((0, 99),(0, 99),(0, 99),(0, 99),(0, 99))
x0 = np.array(lst)
# 没有newton-cg, dogleg, 'trust-ncg', 'trust-exact', 'trust-krylov', 'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'COBYLA'
methods = [ 'L-BFGS-B', 'TNC', 'SLSQP', 'trust-constr']
for method in methods:
    print("|______________________________________________________|")
    print(method)
    res = minimize(fun(), x0, method=method, bounds=bounds, constraints=cons)
    print(res)

|______________________________________________________|
L-BFGS-B
      fun: -106227.0
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-189.99853637, -195.99974621, -590.99838836, -790.99991126,
       -394.00038077])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 3
   status: 0
  success: True
        x: array([99., 99., 99., 99., 99.])
|______________________________________________________|
TNC
     fun: -96624.0
     jac: array([-189.99853637,    1.99943315, -590.99838836, -790.99991126,
       -394.00038077])
 message: 'Local minimum reached (|pg| ~= 0)'
    nfev: 15
     nit: 5
  status: 0
 success: True
       x: array([99.,  0., 99., 99., 99.])
|______________________________________________________|
SLSQP
     fun: -42677.56889637987
     jac: array([   8.        ,    2.        , -197.        , -791.        ,
        -52.13330078])
 message: 'Optimization terminated successfully.'
    nfev: 25
     nit: 2
    n